In [ ]:
# adapting function for XLSX from public records request #23-51

import pandas as pd

fn = 'CulverCity_RentalRegistry_PublicRecordsRequest.xlsx'
adapt = pd.read_excel(fn)
adapt = adapt.rename(columns={
    'Property Address':'AddressLine1',
    'Property City':'AddressLine2',
    'Owner Name':'Owner',
    'Owner Email':'Email',
    'CertOfOccuDate':'OccDate',
    'YearBuilt':'YearBuilt',
    'UnitNumb':'UnitNumb',
    'UnitStatusDescription':'Unit Status',
    'NumofBeds':'NumofBeds',
    'NumOfBaths':'NumOfBaths',
    'Section8Tenant':'Section8',
    'EmcumberedUnit':'EmcumberedUnit',
    'ReportDate':'PastRentReportDate',
    'Amount':'RentAmount'})

adapt['OccDate'] = adapt['OccDate'].replace("<N/A>",None)
adapt['OccDate'] = pd.to_datetime(adapt['OccDate'])

adapt['PastRentReportDate'] = adapt['PastRentReportDate'].replace("<N/A>",None)
adapt['PastRentReportDate'] = pd.to_datetime(adapt['PastRentReportDate'])

adapt['RentAmount'] = adapt['RentAmount'].replace("<N/A>",None)
adapt['RentAmount'] = adapt['RentAmount'].replace('[\$,]', '', regex=True).astype(float)
adapt['RentAmount'] = pd.to_numeric(adapt['RentAmount'])

adapt['EmcumberedUnit'] = adapt['EmcumberedUnit'].replace({"YES":"Yes","NO":"No"})
adapt['Section8'] = adapt['Section8'].replace({"YES":"Yes","NO":"No"})

adapt['Unit Status'] = adapt['Unit Status'].replace("Section 8","Rented")

adapt['Owner'] = adapt['Owner'].replace("<N/A>","")
adapt['Email'] = adapt['Email'].replace("<N/A>","")

adapt.to_excel('Rent Registry_PRA 23-51.xlsx')

In [194]:
# run this to read in the rent registry XLSX file

import pandas as pd
import numpy as np
import usaddress
import datetime
from slugify import slugify

def slugUnit(s):
    return slugify(str(s))

def cleanOwnerName(s):
    if pd.isna(s):
        return ''
    return s.removeprefix(',').strip()

notInCulver =[
    '3824 KEYSTONE AVE # 1',
    '4037 MCLAUGHLIN AVE UNIT 2',
    '12040 ALBERTA DR',
    '11966  CULVER DR  ',
    '373 PERSHING DR',
    '4524 S SLAUSON AVE',
    '250 E 1ST ST # 402',
    '4806 SAWTELLE BLVD',
    '20929 VENTURA BOULEVARD #47-537',
    '8839 N CEDAR AVE',
    'Do not own a rental property in Culver City',
    '550 W 127TH ST # 330',
    '11801 MCDONALD ST.',
    '8839 N CEDAR AVE 212',
]
def inCulver(s):
    return s not in notInCulver

addressDict = {
    '10144 CULVER CITY':'10144 CULVER BLVD',
    '3959 AND 3961 BENTLEY AVE':'3959 BENTLEY AVE',
    '4222 & 4224 JASMINE AVE':'4222 JASMINE AVE',
    '4070 LAFAYETTE PL # House B':'4070 LAFAYETTE PL',
    '4350-68 OVERLOAD AVE':'4350 OVERLAND AVE',
    'Do not own a rental property in Culver City':'',
    '5324 30 KINSTON AVE':'5324 KINSTON AVE',
    '4222-4224 JASMINE AVE':'4222 JASMINE AVE',
    '10761, 10763 NORTHGATE ST':'10761 NORTHGATE ST',
    '10761,10763 NORTHGATE ST':'10761 NORTHGATE ST',
    '4350-68 OVERLAND AVE':'4350 OVERLAND AVE',
    '4251 59 DUQUESNE AVE':'4251 DUQUESNE AVE',
    '4116 CHARLES AVE':'4114 CHARLES AVE',
    '3845 WESTWOOD BLVD':'3847 WESTWOOD BLVD',
    '4132 IRVING PL':'4130 IRVING PL',
    '4132  IRVING PL  ':'4130 IRVING PL',
    '4242 LINCOLN AVE':'4240 LINCOLN AVE',
    '3446 SHERBOURNE DR':'3444 SHERBOURNE DR',
    '4121 LA SALLE AVE':'4119 LA SALLE AVE',
    '4175 JASMINE AVE':'4173 JASMINE AVE',
    '4175  JASMINE AVE  ':'4173 JASMINE AVE',
    '4916 ST LOUIS CT':'4916 SAINT LOUIS CT',
    '4237 LA SALLE AVE':'4235 LA SALLE AVE',
    '4237  LA SALLE AVE  ':'4235 LA SALLE AVE',
    '4162 LAFAYETTE PL':'4160 LAFAYETTE PL',
    '12036 WAGNER ST':'12036 WASHINGTON BLVD',
    '10115-10117 BRADDOCK DR' : '10115 BRADDOCK DR',
'10712 OREGON AVE' : '10710 OREGON AVE',
'10729/10731 NORTHGATE ST' : '10729 NORTHGATE ST',
'10743 JEFFERSON BLVD' : '10739 JEFFERSON BLVD',
'10739-10743 JEFFERSON BLVD' : '10739 JEFFERSON BLVD',
'10742 GALVIN ST' : '10740 GALVIN ST',
'10755 10757 GALVIN ST' : '10755 GALVIN ST',
'10763  GALVIN ST  ' : '10761 GALVIN ST',
'10761-10763 GALVIN ST' : '10761 GALVIN ST',
'10770-10772 GALVIN ST' : '10770 GALVIN ST',
'10771-10773 JEFFERSON BLVD' : '10771 JEFFERSON BLVD',
'10775-10779 JEFFERSON BLVD' : '10775 JEFFERSON BLVD',
'10784  GALVIN ST  ' : '10782 GALVIN ST',
'10784-86 NORTHGATE ST.' : '10784 NORTHGATE ST',
'10787 NORTHGATE ST' : '10785 NORTHGATE ST',
'10838 WASHINGTON BLVD' : '10836 WASHINGTON BLVD',
'10959 73 CULVER BLVD' : '10959 CULVER BLVD',
'10979 87 CULVER BLVD' : '10979 CULVER BLVD',
'10984 CULVER BLVD' : '10982 CULVER BLVD',
'10993 99 CULVER BLVD' : '10993 CULVER BLVD',
'11321 WASHINGTON BLVD' : '11323 WASHINGTON BLVD',
'11370 VENICE BLVD' : '11368 VENICE BLVD',
'11801 MCDONALD ST.' : '11801 MCDONALD ST',
'12022 12024 W WASHINGTON BLVD' : '12022 WASHINGTON BLVD',
'12022-12024 W WASHINGTON BLVD, CULVER CITY, CA 900' : '12022 WASHINGTON BLVD',
'12821 W WASHINGTON BLVD' : '12813 WASHINGTON BLVD',
'2639 FAIRFAX AVE' : '2637 FAIRFAX AVE',
'3252 HUTCHISON AVE' : '3250 HUTCHISON AVE',
'3311-3313 MCMANUS AVE' : '3313 MCMANUS AVE',
'3316 HELMS AVE' : '3314 HELMS AVE',
'3335  CAROLINE AVE  ' : '3333 CAROLINE AVE',
'3406 FAY AVE' : '3404 FAY AVE',
'3420 FAY AVE' : '3418 FAY AVE',
'3427 CATTARAUGUS AVE' : '3425 CATTARAUGUS AVE',
'3436 SHERBOURNE DR' : '3434 SHERBOURNE DR',
'3447 3449 MCMANUS AVE' : '3447 MCMANUS AVE',
'3543/3545 HELMS AVE' : '3545 HELMS AVE',
'3573 HELMS AVE' : '3571 HELMS AVE',
'3590  SCHAEFER ST  ' : '3588 SCHAEFER ST',
'3827 HURON AVE UNIT 2' : '3827 HURON AVENUE',
'3832-34 HURON AVENUE' : '3832 HURON AVE',
'3833  HURON AVE  UNIT 2' : '3833  HURON AVENUE',
'3833  HURON AVE  UNIT 4' : '3833 HURON AVENUE',
'3838 TILDEN AVE' : '3838 TILDEN AVENUE',
'3844 BENTLEY AVE' : '3842 BENTLEY AVE',
'3850 HURON AVE' : '3848 HURON AVE',
'3851 HURON AVE' : '3853 HURON AVE',
'3921-3923 HURON AVE' : '3921 HURON AVE',
'3944-46 HURON AVE' : '3944 HURON AVE',
'3976 TILDEN AVE' : '3976 TILDEN AVENUE',
'4024 HIGUERA ST' : '4022 HIGUERA ST',
'4023 WADE ST' : '4023 WADE STREET',
'90232 4033 HARTER AVE' : '4023 HARTER AVE',
'4043 4047 IRVING PL' : '4043 IRVING PLACE',
'4043, 45, 47 VAN BUREN PL' : '4043 VAN BUREN PL',
'4045  TILDEN AVE  ' : '4045  TILDEN AVENUE',
'4049 LA SALLE AVE' : '4045 LA SALLE AVE',
'4050  LINCOLN AVE  ' : '4046 LINCOLN AVE',
'4048  LINCOLN AVE  ' : '4048 LINCOLN AVENUE',
'4055 LAFAYETTE PLACE' : '4055 LAFAYETTE PL',
'4064 LA SALLE AVE' : '4062 LA SALLE AVE',
'4071 4073 IRVING PL' : '4071 IRVING PL',
'4072 4074 LA SALLE AVE' : '4072 LA SALLE AVE',
'4074  HURON AVE  ' : '4072 HURON AVE',
'4077 HURON AVE' : '4075 HURON AVE',
'4115 HURON AVE' : '4113 HURON AVE',
'4118-4120 LINCOLN AVE' : '4118 LINCOLN AVE',
'4122  CHARLES AVE  ' : '4120 CHARLES AVE',
'4121-4123 MILTON AVE' : '4121 MILTON AVE',
'4125 JASMINE AVE' : '4123 JASMINE AVE',
'4130 WADE ST' : '4128 WADE ST',
'4132 LA SALLE AVE' : '4130 LA SALLE AVE',
'4136 DUQUESNE AVE' : '4134 DUQUESNE AVE',
'4141 LA SALLE AVE' : '4139 LA SALLE AVE',
'4145 LA SALLE AVE' : '4143 LA SALLE AVE',
'4143-47 LA SALLE AVE' : '4143 LA SALLE AVE',
'4151-4153 DUQUESNE AVE' : '4151 DUQUESNE AVE',
'4154/56 JASMINE AVE' : '4154 JASMINE AVE',
'4163 MADISON AVE' : '4161 MADISON AVE',
'4163-4165 LINCOLN AVE' : '4163 LINCOLN AVE',
'4166  LINCOLN AVE  ' : '4164 LINCOLN AVE',
'4170 DUQUESNE AVE' : '4168 DUQUESNE AVE',
'4174 4170 INCE BLVD' : '4170 INCE BLVD',
'4180, 4182 IRVING PL' : '4180 IRVING PL',
'4186  LINCOLN AVE  ' : '4184 LINCOLN AVE',
'9950  FARRAGUT DR  ' : '4205 MADISON AVE',
'4211-4213 BALDWIN AVE' : '4211 BALDWIN AVE',
'4215 JACKSON AVE' : '4213 JACKSON AVE',
'4212 LA SALLE AVE' : '4214 LA SALLE AVE',
'4225 DUQUESNE AVE' : '4223 DUQUESNE AVE',
'4226 MADISON AVE' : '4224 MADISON AVE',
'4245 SAWTELLE BLVD' : '4241 SAWTELLE BLVD',
'4252 LINCOLN AVE' : '4250 LINCOLN AVE',
'4255 LA SALLE AVE' : '4253 LA SALLE AVE',
'4267 BALDWIN AVE' : '4265 BALDWIN AVE',
'4281, 4283 REVERE PL' : '4281 REVERE PL',
'4904 TARA TERRACE' : '4904 TARA TER',
'5314 5320 KINSTON AVE' : '5314 KINSTON AVE',
'5351-5357 KINSTON AVE' : '5351 KINSTON AVE',
'5361-5367 KINSTON AVE' : '5361 KINSTON AVE',
'5403-5409 KINSTON AVE' : '5403 KINSTON AVE',
'5415-5421 KINSTON AVE' : '5415 KINSTON AVE',
'5418  KINSTON AVE  ' : '5416 KINSTON AVE',
'5582 KINSTON AVE' : '5576 KINSTON AVE',
'5627-33 KINSTON AVE' : '5627 KINSTON AVE',
'6300 GREEN VALLEY CIR # LEASING' : '6300 GREEN VALLEY CIR',
'8905  HUBBARD ST  ' : '8903 HUBBARD ST',
'9046  LUCERNE AVE  ' : '9044 LUCERNE AVE',
'9055 LUCERNE AVE # A' : '9053 LUCERNE AVE',
'9630 BRADDOCK DR' : '9628 BRADDOCK DR',
'4038 WADE ST # 1' : '4040 WADE ST',
'5019 OVERLAND AVE' : '5021 OVERLAND AVE',
'3902 MIDWAY AVE' : '3900 MIDWAY AVE',
'3904 MIDWAY AVE' : '3900 MIDWAY AVE',
'11914 ATLANTIC AVE' : '11900 ATLANTIC AVE',
}

def cleanAddressLine1(a):
    if pd.isna(a):
        return ''
    if isinstance(a,str) and a in addressDict.keys():
        return addressDict[a]
    if isinstance(a,str) and a in ['3959 AND 3961 BENTLEY AVE','4070 LAFAYETTE PL # House B','4222 & 4224 JASMINE AVE']:
        return '3959 BENTLEY AVE'
    if isinstance(a,datetime.datetime):
        return '4051 JACKSON AVE'
    tagged_address, address_type= usaddress.tag(str(a))
    if 'StreetNamePostType' not in tagged_address.keys():
        return ' '.join([tagged_address['AddressNumber'], tagged_address['StreetName']])
    else:
        return ' '.join([tagged_address['AddressNumber'], tagged_address['StreetName'], tagged_address['StreetNamePostType']])

def cleanAssessor(a, b):
    tagged_address, address_type= usaddress.tag(str(b)+" "+str(a))
    if 'AddressNumber' not in tagged_address.keys():
        return ''
    elif 'StreetNamePostType' not in tagged_address.keys():
        return ' '.join([tagged_address['AddressNumber'], tagged_address['StreetName']])
    else:
        return ' '.join([tagged_address['AddressNumber'], tagged_address['StreetName'], tagged_address['StreetNamePostType']])

la_assessor = pd.read_csv("la_assessor.csv",low_memory=False,usecols=['RollYear','UseCodeDescChar2','Roll_HomeOwnersExemp','SitusStreet','SitusHouseNo','AIN'],dtype={'UseCodeDescChar2':'category'})
la_assessor = la_assessor[(la_assessor['RollYear'] == 2022)]
#la_assessor = la_assessor[(la_assessor['RollYear'] == 2022) & ((la_assessor['UseCodeDescChar2'] == "Single Family Residence") | ((la_assessor['UseCodeDescChar2'] == "Double, Duplex, or Two Units") & (la_assessor['Roll_HomeOwnersExemp'] != 0)))][['SitusStreet','SitusHouseNo']]
#la_assessor = la_assessor[((la_assessor['UseCodeDescChar2'] == "Single Family Residence") | ((la_assessor['UseCodeDescChar2'] == "Double, Duplex, or Two Units") & (la_assessor['Roll_HomeOwnersExemp'] != 0)))]


def useAIN(ain, number, street):
    la_assessor.at[la_assessor[la_assessor['AIN'] == ain].index[0], 'SitusHouseNo'] = number
    la_assessor.at[la_assessor[la_assessor['AIN'] == ain].index[0], 'SitusStreet'] = street
ainTuples = [
    (4232015039, '12116', 'WASHINGTON BLVD'),
    (4232006019, '12320', 'WASHINGTON PL'),
    (4207010048, '4058', 'MADISON AVE'),
    (4312031014,'8809','WASHINGTON BLVD'),
    (4232008019,'12228','WASHINGTON PL'),
    (4231022003,'4109','NEOSHO AVE'),
    (4312023013,'6105','WASHINGTON BLVD'),
    (4208015015,'3900', 'MIDWAY AVE'),
    (4207012013,'10115','BRADDOCK DR'),
    (4134013001, '6300','GREEN VALLEY CIR'),
    (4203002043, '10736','JEFFERSON BLVD'),
    (4207026017,'10156','FARRAGUT DR'),
    (4207010005,'4000','MADISON AVE'),
    (4208026019,'10949','ALETTA AVE')

]
for t in ainTuples:
    useAIN(t[0],t[1],t[2])

la_assessor.at[la_assessor[la_assessor['AIN'] == 4232015039].index[0], 'SitusHouseNo'] = '12116'
la_assessor.at[la_assessor[la_assessor['AIN'] == 4232015039].index[0], 'SitusStreet'] = 'WASHINGTON BLVD'
la_assessor['SitusAddress'] = la_assessor.apply(lambda x: cleanAssessor(x.SitusStreet,x.SitusHouseNo), axis=1).apply(slugify)
la_assessor = la_assessor.sort_values(by='Roll_HomeOwnersExemp',ascending=False).drop_duplicates(subset=['SitusAddress'],keep='first')

#def singleFamily(a):
#    return (la_assessor['SitusAddress'].eq(a)).any()

fn = 'Rent Registry_PRA 22-496.xlsx'
fn = 'Rent Registry_PRA 23-51.xlsx'
#fn = 'test.xlsx'
ccrr = pd.read_excel(fn,dtype={'Unit Status':'category','NumofBeds':'category','NumOfBaths':'category','Section8':'category','EmcumberedUnit':'category'})
ccrr['OriginalAddress'] = ccrr['AddressLine1']
ccrr['inCulver'] = ccrr['AddressLine1'].apply(inCulver)
ccrr = ccrr[ccrr['inCulver']]
ccrr['AddressLine1'] = ccrr['AddressLine1'].apply(cleanAddressLine1)
ccrr['Owner'] = ccrr['Owner'].apply(cleanOwnerName)
ccrr = ccrr.sort_values(by='PastRentReportDate',ascending=False).reset_index()
ccrr['slug'] = ccrr['AddressLine1'].apply(slugify)
ccrr['UnitNumb'] = ccrr['UnitNumb'].apply(slugUnit)

ccrr = ccrr.merge(la_assessor,how='left',left_on=['slug'],right_on=['SitusAddress'])

ccrr['SFR_Condo'] = ccrr['UseCodeDescChar2'] == "Single Family Residence"
ccrr['DuplexOO'] = (ccrr['UseCodeDescChar2'] == "Double, Duplex, or Two Units") & (ccrr['Roll_HomeOwnersExemp'] != 0)
#ccrr.to_excel('test_join_assessor.xlsx')

c:\Users\jones\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\parsers\base_parser.py:761: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  cats = Index(values).unique().dropna()
c:\Users\jones\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\parsers\base_parser.py:761: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  cats = Index(values).unique().dropna()


In [195]:
# process ccrr to apply rent thresholds and add "problem" fields

rent_thresholds = pd.DataFrame({
    'start_date':[
        np.datetime64("2020-10-30"),
        np.datetime64("2021-01-01"),
        np.datetime64("2021-10-01"),
        np.datetime64("2021-12-01"),
        np.datetime64("2022-01-01"),
        np.datetime64("2022-02-01"),
        np.datetime64("2022-03-01"),
        np.datetime64("2022-04-01"),
        np.datetime64("2022-05-01"),
    ],
    'end_date':[
        np.datetime64("2020-12-31"),
        np.datetime64("2021-09-30"),
        np.datetime64("2021-11-30"),
        np.datetime64("2021-12-31"),
        np.datetime64("2022-01-31"),
        np.datetime64("2022-02-28"),
        np.datetime64("2022-03-31"),
        np.datetime64("2022-04-30"),
        np.datetime64("2023-04-30"),
    ],
    'threshold':[
        .0225,
        .02,
        .0225,
        .0250,
        .03,
        .0350,
        .0375,
        .0450,
        .05,
    ]
})

def applyRentThreshold(date,increase):
    thresholds = rent_thresholds[(date >= rent_thresholds['start_date']) & (date <= rent_thresholds['end_date'])]
    t = 0
    if thresholds.shape[0] == 1:
        t = thresholds['threshold'].iloc[0]
    elif thresholds.shape[0] > 1:
        raise Exception("rent_thresholds array has overlapping date intervals")
    else:
        t = 0.05
    return increase > t

def getThreshold(date):
    thresholds = rent_thresholds[(date >= rent_thresholds['start_date']) & (date <= rent_thresholds['end_date'])]
    t = 0
    if thresholds.shape[0] == 1:
        t = thresholds['threshold'].iloc[0]
    elif thresholds.shape[0] > 1:
        raise Exception("rent_thresholds array has overlapping date intervals")
    else:
        t = 0.05
    return t

def neverVacant(slug,unit,startDate, endDate):
    return ccrr[
        (ccrr['slug'] == slug) &
        (ccrr['UnitNumb'] == unit) &
        (ccrr['PastRentReportDate'] >= startDate) &
        (ccrr['PastRentReportDate'] <= endDate) &
        (ccrr['Unit Status'] == 'Vacant')
    ].shape[0] == 0

rent_jumps = ccrr.merge(ccrr,on=['slug','UnitNumb'])[['slug','UnitNumb','SFR_Condo_x','YearBuilt_x','RentAmount_x','RentAmount_y','PastRentReportDate_x','PastRentReportDate_y']]
rent_jumps['diff_days'] = (rent_jumps['PastRentReportDate_x'] - rent_jumps['PastRentReportDate_y']) / np.timedelta64(1, 'D')
rent_jumps['diff_years'] = (rent_jumps['PastRentReportDate_x'] - rent_jumps['PastRentReportDate_y']) / np.timedelta64(1, 'Y')
rent_jumps['increase'] = rent_jumps['RentAmount_x'] - rent_jumps['RentAmount_y']
rent_jumps['perc_increase'] = rent_jumps['increase'] / rent_jumps['RentAmount_y']
rent_jumps = rent_jumps[
    (rent_jumps['diff_days'] > 0)
    ]
single_jumps = rent_jumps.sort_values(by='PastRentReportDate_y',ascending=False).drop_duplicates(subset=['slug','UnitNumb','PastRentReportDate_x'],keep='first')
ccrr = ccrr.merge(single_jumps[['slug','UnitNumb','PastRentReportDate_x','perc_increase']],how='left',left_on=['slug','UnitNumb','PastRentReportDate'],right_on=['slug','UnitNumb','PastRentReportDate_x'])

cumu_jumps = rent_jumps[    
    (rent_jumps['RentAmount_x'] > 2000) & 
    (rent_jumps['RentAmount_y'] > 2000) & 
    (rent_jumps['RentAmount_x'] < 10000) & 
    (rent_jumps['RentAmount_y'] < 10000) &
    (rent_jumps['YearBuilt_x'] < 1995) & 
    (rent_jumps['PastRentReportDate_x'] >= np.datetime64("2020-10-30")) &
    (rent_jumps['diff_years'] <= 1) &
    (rent_jumps['SFR_Condo_x'] == False)
]
cumu_jumps['applyThreshold'] = cumu_jumps.apply(lambda x: applyRentThreshold(x.PastRentReportDate_x,x.perc_increase), axis=1)
cumu_jumps = cumu_jumps[cumu_jumps['applyThreshold']]
cumu_jumps['neverVacant'] = cumu_jumps.apply(lambda x: neverVacant(x.slug,x.UnitNumb,x.PastRentReportDate_x,x.PastRentReportDate_y), axis=1)
cumu_jumps = cumu_jumps[cumu_jumps['neverVacant']]
cumu_jumps = cumu_jumps.sort_values(by='perc_increase',ascending=False).drop_duplicates(subset=['slug','UnitNumb'],keep="first")
cumu_jumps['prob'] = True
cumu_jumps['threshold'] = cumu_jumps['PastRentReportDate_x'].apply(getThreshold)
#prob_jumps = cumu_jumps.sort_values(by='perc_increase',ascending=False).groupby(["slug","UnitNumb","PastRentReportDate_x"],as_index=False)["perc_increase"].max()
#prob_jumps['prob'] = True


ccrr = ccrr.merge(cumu_jumps[['slug','UnitNumb','PastRentReportDate_x','perc_increase','PastRentReportDate_y','prob','threshold']],how='left',left_on=['slug','UnitNumb','PastRentReportDate'],right_on=['slug','UnitNumb','PastRentReportDate_x']).rename(columns={'perc_increase_x':'perc_increase','perc_increase_y':'cumulative_increase'})
ccrr['prob'] = ccrr['prob'].replace(np.nan,False)
ccrr = ccrr.groupby(['slug','UnitNumb'],as_index=False)['prob'].any().merge(ccrr,how='right',on=['slug','UnitNumb']).rename(columns={'prob_x':'unit_problem','prob_y':'date_problem'})
ccrr = ccrr.groupby(['slug'],as_index=False)['unit_problem'].any().merge(ccrr,how='right',on=['slug']).rename(columns={'unit_problem_x':'address_problem','unit_problem_y':'unit_problem'})
#ccrr_rent_jumps.to_excel('rent_jumps.xlsx')

<ipython-input-195-0065c60a78e7>:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cumu_jumps['applyThreshold'] = cumu_jumps.apply(lambda x: applyRentThreshold(x.PastRentReportDate_x,x.perc_increase), axis=1)


In [ ]:
late_occupancy = ccrr[(ccrr['date_problem']) & (ccrr['OccDate'] > np.datetime64('1994-12-31'))]
late_occupancy.to_excel("check_these.xlsx")

In [ ]:
la = pd.read_csv("la_assessor.csv",low_memory=False)
la = la[la['RollYear'] == 2022][['SitusStreet','SitusHouseNo','UseCodeDescChar2','Roll_HomeOwnersExemp','YearBuilt','EffectiveYearBuilt']]
la['SitusAddress'] = la.apply(lambda x: cleanAssessor(x.SitusStreet,x.SitusHouseNo), axis=1)
la['SitusAddress'] = la['SitusAddress'].apply(slugify)
date_problem_units = ccrr[ccrr['date_problem']]
date_problem_units = date_problem_units.merge(la,how='left',left_on=['slug'],right_on=['SitusAddress'])
date_problem_units.to_excel("date_problem_units.xlsx")

In [168]:
# geocode addresses in ccrr dataframe

from geocodio import GeocodioClient
import time

key = 'd8f821033f22f02a2d80862a2286d6d2803dd2a'
client = GeocodioClient(key)

# run this to geocode all the addresses in the rent registry
# make sure you use the exact same usaddress format as below in the "address" variable
existing_geocode = pd.read_csv('cc_geocoded.csv')
addresses = ccrr.drop_duplicates(['AddressLine1'])['AddressLine1']
#addresses = addresses.apply(cleanAddressLine1).drop_duplicates()
headers = ['address','lat','lon']
geocoded = existing_geocode.copy()
l = 0
addresses_to_geocode = []
for address in addresses:
    if address == '':
        continue
    elif address in existing_geocode['address'].values:
        continue
    addresses_to_geocode.append(address)

    l += 1
    if l > 10000:
        break

if len(addresses_to_geocode) > 0:
    geocoded_location = client.batch_geocode([s + ', CULVER CITY, CA' for s in addresses_to_geocode])
    coords = geocoded_location.coords
    #coords = []
    #for address in addresses_to_geocode:
    #    lat = l*20
    #    lon = l*2+3
    #    l += 1
    #    coords.append((lat,lon))
    #print(coords)

    i = 0
    for address in addresses_to_geocode:
        if coords[i] == None:
            geocoded = pd.concat([geocoded,pd.DataFrame([[address, '', '']],columns=headers)])
        else:
            geocoded = pd.concat([geocoded,pd.DataFrame([[address, coords[i][0], coords[i][1]]],columns=headers)])
        i += 1

    existing_geocode.to_csv('cc_geocoded_backup-' + time.strftime("%Y%m%d-%H%M%S") + '.csv',index=False)
    geocoded.to_csv('cc_geocoded.csv',index=False)
    

In [196]:
# THIS ONE CREATES THE FLAT GEOJSON "geo" file

import datetime
import time
import orjson
import math
from slugify import slugify

flat_json = dict()
flat_json['type'] = 'FeatureCollection'
flat_features = []
a = []

existing_geocode = pd.read_csv('cc_geocoded.csv')
registered_addresses = []
owners = dict()

for index, row in ccrr.drop_duplicates(['AddressLine1','UnitNumb']).iterrows():
    if pd.isna(row['AddressLine1']):
        continue

    line1 = row['AddressLine1']
    line2 = row['AddressLine2']
    unit = row['UnitNumb']
    beds = row['NumofBeds']
    baths = row['NumOfBaths']
    built = row['YearBuilt']
    rent = row['RentAmount']
    if rent == "<N/A>":
        rent = ""
    if pd.isna(row['PastRentReportDate']):
        rentdate = ''
    elif isinstance(row['PastRentReportDate'],str):
        rentdate = row['PastRentReportDate']
    else:
        rentdate = row['PastRentReportDate'].strftime('%Y/%m/%d')
    encumbered = row['EmcumberedUnit']
    status = row['Unit Status']
    slug = row['slug']
    sfr_exempt = row['SFR_Condo']
    perc_increase = row['perc_increase']
    date_problem = row['date_problem']
    unit_problem = row['unit_problem']
    address_problem = row['address_problem']


    if pd.isna(row["Owner"]):
        owner = ''
    else:
        owner = row['Owner'].removeprefix(', ').removeprefix(',').title()

    if pd.isna(row['Email']):
        email = ''
    else:
        email = row['Email']
    occdate = row['OccDate']
    section = row['Section8']
    
    address = line1
    if address == '':
        continue
    
    #create flat_json with a feature for every unit
    flat = dict()
    flat['type'] = 'Feature'

    geo = existing_geocode[existing_geocode['address'] == address]
    flat['geometry'] = {
        'type':'Point',
        'coordinates':[geo['lon'].values[0].item(),geo['lat'].values[0].item()]
    }

    address = address.title()
    flat['properties'] = {
        'address':address,
        'unit':unit,
        'beds':beds,
        'baths':baths,
        'built':built,
        'rent':rent,
        'rentdate':rentdate,
        'encumbered':encumbered,
        'status':status,
        'slug':slug,
        'owner':owner,
        'email':email,
        #'occdate':occdate,
        'section':section,
        'sfr_exempt':sfr_exempt,
        'perc_increase':perc_increase,
        'date_problem':date_problem,
        'unit_problem':unit_problem,
        'address_problem':address_problem,
    }

    if slug not in owners.keys():
        owners[slug] = []
    if owner not in owners[slug]:
        owners[slug].append(owner)

    registered = False
    if rent == '':
        registered = False
    else:
        registered = True
    #for key in flat['properties'].keys():
    #    if flat['properties']['address'] == '11460 WASHINGTON BLVD':
    #        a.append(flat['properties'][key])
    #    if key == 'address' or key == 'slug':
    #        continue
    #    if flat['properties'][key] != None and flat['properties'][key] != '' and flat['properties'][key] != 'null' and (isinstance(flat['properties'][key],str) or isinstance(flat['properties'][key],datetime.datetime) or not math.isnan(flat['properties'][key])):
    #        registered = True
    #        break
    flat['properties']['registered'] = registered
    if registered:
        registered_addresses.append(address)
    flat_features.append(flat)

# if an address is clearly registered, remove any rows for that address that are not registered!!! (e.g., 3801 College)
flat_features = [x for x in flat_features if not(x['properties']['address'] in registered_addresses and not x['properties']['registered'])]

for i in range(len(flat_features)):
    if len(owners[flat_features[i]['properties']['slug']]) > 1:
        flat_features[i]['properties']['multiple_owners'] = True
    else:
        flat_features[i]['properties']['multiple_owners'] = False

flat_json['features'] = flat_features

with open("ccrr-geo-" + time.strftime("%Y%m%d-%H%M%S") + ".json", "wb") as f:
    f.write(orjson.dumps(flat_json))

In [197]:
# from flat_json, create JSON of owners sorted by number of occurrences "owner_counts" file

def title(s):
    return s.title()

owners = []

for f in flat_json['features']:
    owners.append(f['properties']['owner'])
owners = pd.DataFrame(owners, columns=['index']).replace('', np.nan).dropna()
owners = owners.value_counts().reset_index(name='count').sort_values(['count'], ascending=False)
owners = pd.DataFrame(owners)
owners['index'] = owners['index'].apply(title)
owners['label'] = owners['index']
owners = owners.rename(columns={'index':'value'})
owners.to_json('owner_counts.json',orient="records")

In [198]:
# from flat_json, create JSON of addresses sorted by number of occurrences "address_counts" file

def title(s):
    return s.title()

addresses = []

for f in flat_json['features']:
    addresses.append(f['properties']['address'])

addresses = pd.DataFrame(addresses, columns=['index']).replace('', np.nan).dropna()
addresses = addresses.value_counts().reset_index(name='count').sort_values(['count'], ascending=False)
addresses = pd.DataFrame(addresses)
addresses['index'] = addresses['index'].apply(title)
addresses['value'] = addresses['index']
addresses = addresses.rename(columns={'index':'label'})
addresses.to_json('address_counts.json',orient="records")

In [199]:
# turn flat_json into csv file of properties "flat" CSV file (not used in production app, only for reference)

headers = ['address',
        'unit',
        'beds',
        'baths',
        'built',
        'rent',
        'rentdate',
        'encumbered',
        'status',
        'registered',
        'slug',
        'owner',
        'email',
        #'occdate',
        'section',
        'multiple_owners',
        'sfr_exempt',
        'perc_increase',
        'date_problem',
        'unit_problem',
        'address_problem',]
flat_df = pd.DataFrame(columns=headers)


for f in flat_json['features']:
    row = []
    for i in range(len(headers)):
        row.append(f['properties'][headers[i]])
    flat_df = pd.concat([flat_df,pd.DataFrame([row],columns=headers)])
flat_df.to_csv("ccrr-flat-" + time.strftime("%Y%m%d-%H%M%S") + ".csv")

In [200]:
# turn flat_json into json file of properties (NOT GeoJSON!) used for "data" file

headers = ['address',
        'unit',
        'beds',
        'baths',
        'built',
        'rent',
        'rentdate',
        'encumbered',
        'status',
        'registered',
        'slug',
        'owner',
        'email',
        #'occdate',
        'section',
        'multiple_owners',
        'sfr_exempt',
        'perc_increase',
        'date_problem',
        'unit_problem',
        'address_problem',]
flat_df = pd.DataFrame(columns=headers)


for f in flat_json['features']:
    row = []
    for i in range(len(headers)):
        row.append(f['properties'][headers[i]])
    flat_df = pd.concat([flat_df,pd.DataFrame([row],columns=headers)])
flat_df.to_json("ccrr-data-" + time.strftime("%Y%m%d-%H%M%S") + ".json",orient="records")

In [203]:
# make JSON file of historical rents used for "hist" file
import time
#TODO: remove any row that has null in date or rent
#TODO: remove any row that has identical date and rent as another row (within the same slug/unit pair)

def stringDates(d):
    if pd.isna(d):
        return ''
    elif isinstance(d,str):
        return d
    else:
        return d.strftime('%Y/%m/%d')

historical_rents = ccrr[['slug','UnitNumb','PastRentReportDate','RentAmount','perc_increase','date_problem','PastRentReportDate_y','cumulative_increase','threshold','Unit Status']]
historical_rents = historical_rents.rename(columns={'UnitNumb':'unit','PastRentReportDate':'rentdate','RentAmount':'rent'})
historical_rents['rentdate'] = historical_rents['rentdate'].apply(stringDates)
historical_rents['PastRentReportDate_y'] = historical_rents['PastRentReportDate_y'].apply(stringDates)
historical_rents['id'] = historical_rents.index
historical_rents.to_json("ccrr-hist-" + time.strftime("%Y%m%d-%H%M%S") + ".json",orient="records",index=True)

In [204]:
ccrr

,slug,address_problem,UnitNumb,unit_problem,index,Column1,AddressLine1,AddressLine2,Owner,Email,...,SitusAddress,SFR_Condo,DuplexOO,PastRentReportDate_x_x,perc_increase,PastRentReportDate_x_y,cumulative_increase,PastRentReportDate_y,date_problem,threshold
0,11605-washington-blvd,False,d,False,21205,21205,11605 WASHINGTON BLVD,"CULVER CITY, CA 90066","TOMINAGE, KIMBERLY",NaN,...,11605-washington-blvd,False,False,2023-02-06,0.064444,NaT,NaN,NaT,False,NaN
1,11605-washington-blvd,False,d,False,21197,21197,11605 WASHINGTON BLVD,"LOS ANGELES, CA 90066-5924","TOMINAGA, KIMBERLY",meadowlarkmgmt@gmail.com,...,11605-washington-blvd,False,False,2023-02-06,0.064444,NaT,NaN,NaT,False,NaN
2,11605-washington-blvd,False,c,False,21195,21195,11605 WASHINGTON BLVD,"LOS ANGELES, CA 90066-5924","TOMINAGA, KIMBERLY",meadowlarkmgmt@gmail.com,...,11605-washington-blvd,False,False,2023-02-06,0.060000,NaT,NaN,NaT,False,NaN
3,11605-washington-blvd,False,b,False,21193,21193,11605 WASHINGTON BLVD,"LOS ANGELES, CA 90066-5924","TOMINAGA, KIMBERLY",meadowlarkmgmt@gmail.com,...,11605-washington-blvd,False,False,2023-02-06,-0.001961,NaT,NaN,NaT,False,NaN
4,11605-washington-blvd,False,a,False,21199,21199,11605 WASHINGTON BLVD,"CULVER CITY, CA 90066","TOMINAGE, KIMBERLY",NaN,...,11605-washington-blvd,False,False,2023-02-06,0.063265,NaT,NaN,NaT,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20988,10771-jefferson-blvd,False,10773,False,20324,20324,10771 JEFFERSON BLVD,"CULVER CITY, CA 90230",,NaN,...,10771-jefferson-blvd,False,False,NaT,NaN,NaT,NaN,NaT,False,NaN
20989,11163-braddock-dr,False,11163,False,20325,20325,11163 BRADDOCK DR,"CULVER CITY, CA 90230-4214",,NaN,...,11163-braddock-dr,True,False,NaT,NaN,NaT,NaN,NaT,False,NaN
20990,4029-sawtelle-blvd,False,4029,False,20580,20580,4029 SAWTELLE BLVD,"CULVER CITY, CA 90066",,NaN,...,4029-sawtelle-blvd,True,False,NaT,NaN,NaT,NaN,NaT,False,NaN
20991,3924-bentley-ave,False,1,False,20581,20581,3924 BENTLEY AVE,"CULVER CITY, CA 90232-3906",,NaN,...,3924-bentley-ave,False,False,NaT,NaN,NaT,NaN,NaT,False,NaN


In [ ]:
max_length = 0
longest_history = []
for index, row in historical_rents.drop_duplicates(['slug','unit']).iterrows():
    subset = historical_rents[(historical_rents['slug'] == row['slug']) & (historical_rents['unit'] == row['unit'])]
    if subset.shape[0] > max_length:
        max_length = subset.shape[0]
        longest_history = [row['slug'], row['unit']]


In [ ]:
print(max_length)
print(longest_history)

In [ ]:
# turn big PDF data into 5 different PDFs, each representing a different set of columns

from PyPDF2 import PdfWriter, PdfReader

inputpdf = PdfReader(open("Rent Registry_PRA 23-51.pdf", "rb"))
pagesper = 452
tables = 5

for i in range(tables):
    output = PdfWriter()
    for j in range(pagesper):
        output.add_page(inputpdf.pages[i*pagesper+j])
    with open("document-page%s.pdf" % i, "wb") as outputStream:
        output.write(outputStream)

In [ ]:
# check if those 5 PDFs all have the same number \n "rows"
import PyPDF2

fns = [
    'document-page0.pdf',
    'document-page1.pdf',
    'document-page2.pdf',
    'document-page3.pdf',
    'document-page4.pdf',
    ]
for fn in fns:
    pdfFileObj = open(fn ,'rb')

    pdfReader = PyPDF2.PdfReader(pdfFileObj)

    pages = len(pdfReader.pages)


    new_lines = 0
    for i in range(pages):
        new_lines += pdfReader.pages[i].extract_text().count('\n')+1
    print(fn,": ",new_lines)
    pdfFileObj.close()

In [ ]:
# take the 5 PDFs and split the newlines, making them into rows

import PyPDF2
import pandas as pd

fns = [
    'document-page0.pdf',
    'document-page1.pdf',
    'document-page2.pdf',
    'document-page3.pdf',
    'document-page4.pdf',
    ]

columns = []

# page_ones = []
for fn in fns:
    col = []
    pdfFileObj = open(fn ,'rb')

    pdfReader = PyPDF2.PdfReader(pdfFileObj)
    for page in pdfReader.pages:
        col += page.extract_text().split('\n')

    # page_ones.append(pdfReader.pages[0].extract_text())
    columns.append(col)
    pdfFileObj.close()

In [ ]:
# take the 2d array created above and change to a pandas df, using regexes to split the columns

# you can run the main functions after this to import the XLSX file and generate GEOJSON
pdf_units = pd.DataFrame()
for col in columns:
    pdf_units[col[0]] = col[1:]

reg_strings = {
    columns[0][0]:r'(?P<AddressLine1>.+)',
    columns[1][0]:r'(?P<Owner>.+)',
    columns[2][0]:r'(?P<Email>\S+\@\S+)?\s*(?P<OccDate>\d?\d/\d?\d/\d{4}|<N/A>|\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3})\s*(?P<YearBuilt>\d{4}|0)',
    columns[3][0]:r'(?P<UnitNumb>.*)\s*(?P<UnitStatus>Rented|Vacant|Section 8*)\s*(?P<NumofBeds>\d*)\s*(?P<NumOfBaths>\S*)',
    columns[4][0]:r'(?P<Section8>YES|NO*)\s*(?P<EmcumberedUnit>YES|NO*)\s*(?P<PastRentReportDate>\S*)\s*(?P<RentAmount>\S*)'
}

pdf_units[columns[3][0]].str.extract(reg_strings[columns[3][0]]).loc[[3666]]

In [ ]:
# take pdf_units and convert to dataframe, then save as XLSX file


split_units = pd.DataFrame()
for i in range(5):
    split_units = pd.concat([split_units,pdf_units[columns[i][0]].str.extract(reg_strings[columns[i][0]],expand=True)], axis=1)

split_units = split_units.rename(columns={'UnitStatus':'Unit Status'})

split_units['AddressLine2'] = ""

split_units['OccDate'] = split_units['OccDate'].replace("<N/A>",None)
split_units['OccDate'] = pd.to_datetime(split_units['OccDate'])

split_units['PastRentReportDate'] = split_units['PastRentReportDate'].replace("<N/A>",None)
split_units['PastRentReportDate'] = pd.to_datetime(split_units['PastRentReportDate'])

split_units['RentAmount'] = split_units['RentAmount'].replace("<N/A>",None)
split_units['RentAmount'] = split_units['RentAmount'].replace('[\$,]', '', regex=True).astype(float)
split_units['RentAmount'] = pd.to_numeric(split_units['RentAmount'])

split_units['EmcumberedUnit'] = split_units['EmcumberedUnit'].replace({"YES":"Yes","NO":"No"})
split_units['Section8'] = split_units['Section8'].replace({"YES":"Yes","NO":"No"})

split_units['Unit Status'] = split_units['Unit Status'].replace("Section 8","Rented")

split_units['Owner'] = split_units['Owner'].replace("<N/A>","")

split_units.to_excel('Rent Registry_PRA 23-51.xlsx')